# HW 2, Part 2: Scalable algorithms with SQL #



In [3]:
import subprocess, sqlite3, csv, os, sys

# jupyter weirdness; this makes sure that stdout is not buffered
def printNow(s):
    print s
    sys.stdout.flush()

    
def checkFile(f):
    if not (os.path.isfile(f)):
        raise IOError("DB File " + f + " is not here!")


currentDB1 = currentDB2 = "part2test.db"

# for grading dataset
# note that the two DB are different! currentDB1 is smaller in size

if ('CS186GRADER' in os.environ):
    currentDB1 = "part2sampled.db" 
    currentDB2 = "part2.db" 
    
checkFile(currentDB1)
dbQ1 = sqlite3.connect(currentDB1) 
dbQ1.text_factory = str

checkFile(currentDB2)
dbQ2 = sqlite3.connect(currentDB2)
dbQ2.text_factory = str
print(currentDB2)

part2test.db


In [4]:
# Q1
# Degrees of separation
# Expected output: a paths table (as defined), note that we will be checking the original
#                  paths, and not just the average value

def initDegrees():
    dbQ1.executescript("""
        -- create paths table
        DROP TABLE IF EXISTS paths;
        DROP TABLE IF EXISTS pathsLastUpdated;
        DROP INDEX IF EXISTS paths_src_idx;
        DROP INDEX IF EXISTS paths_dst_idx;

        CREATE TABLE paths (
            src varchar(200),
            dst varchar(200),
            length integer,
            PRIMARY KEY (src, dst)
        );
        
        CREATE TABLE pathsLastUpdated (
            src varchar(200),
            dst varchar(200),
            length integer
        );

        CREATE INDEX paths_src_idx ON paths(src);
        CREATE INDEX paths_dst_idx ON paths(dst);
        
    """);
    dbQ1.commit()
    return


def populatePaths():
    dbQ1.executescript("""
      -- initialize paths with the first level connections
      INSERT INTO paths SELECT
        src,
        dst,
        1
      FROM links;      
      
      -- initialized for your convenience the "last udpated"
      INSERT INTO pathsLastUpdated SELECT
        src,
        dst,
        1
      FROM links;
    """);
    
    dbQ1.commit()
    return;


# Please implement the algorithm such that you don't repeat paths that were previously explored
# For example, for graph a->b->c->d, we want to see (a,b), (b,c), (c,d) then (a,c), (b,d), then (a,d)
# This makes a rather significant performance difference. To this end we have set up the 
#   pathsLastUpdated for you to store intermediate results
def getNewPaths():
    dbQ1.executescript("""
        -- this is an intermediate table for you to store temp data
        DROP TABLE IF EXISTS pathsNew;  
        CREATE TABLE pathsNew (
            src varchar(200),
            dst varchar(200),
            length integer
        );    
        
        -- INSERT INTO pathsNew        
            -- TODO (hint: use pathsLastUpdated)
            INSERT INTO pathsNew 
            select l.src , p.dst, 1 + p.length as len   from links l, pathsLastUpdated p
            where l.dst = p.src;
            
        -- updating paths_last_update
        
           delete from pathsLastUpdated;
            
        -- insert newpaths to the Paths_last_update.
        
            insert into pathsLastUpdated
            select * from pathsNew;
            
        -- ;
     """)
    dbQ1.commit()
    return

def updatePathsWithNewpaths():
    dbQ1.executescript("""
        
        
        -- add it to path
        -- INSERT INTO paths
            -- TODO (hint: use pathsNew)
        -- ;
            
            DROP TABLE IF EXISTS paths_t;

            CREATE TABLE paths_t (
            src varchar(200),
            dst varchar(200),
            length integer,
            PRIMARY KEY (src, dst)
            );
            
            insert into paths_t
                select src, dst, min(length) from
                (select * from paths
                union
                select distinct * from pathsNew)
                group by src,dst;
                
            delete from paths;
            
            insert into paths 
            select * from paths_t;
            
            DROP TABLE IF EXISTS paths_t;
            
            
    """)
    dbQ1.commit()
    return


def getPathsCount():
    r = dbQ1.execute("SELECT COUNT(*) FROM paths;").fetchone()
    return r[0]


def getAveragePathsLength():
    r = dbQ1.execute("SELECT avg(length) FROM paths;").fetchone()
    return r[0]


def findDegreeDistribution():
    printNow("initializing degrees.  Calling sqlite...")
    initDegrees()
    printNow("Done, now populating paths.  Calling sqlite...")
    populatePaths()
    printNow("Done, now getting pathcount.  Calling sqlite...")
    oldCount = getPathsCount()
    i = 0
    while True:
        i += 1
        printNow("Beginning iteration #" + str(i) + ".  Calling sqlite ...")
        getNewPaths()
        printNow("Updating paths")
        updatePathsWithNewpaths()
        currentCount = getPathsCount()
        printNow("Done, current path total is: " + str(currentCount))
        if (currentCount == oldCount):
            break
        else:
            oldCount = currentCount

            
    print 'Converged! The average length of the shortest paths is: ', getAveragePathsLength()

findDegreeDistribution()

initializing degrees.  Calling sqlite...
Done, now populating paths.  Calling sqlite...
Done, now getting pathcount.  Calling sqlite...
Beginning iteration #1.  Calling sqlite ...
Updating paths
Done, current path total is: 15
Beginning iteration #2.  Calling sqlite ...
Updating paths
Done, current path total is: 18
Beginning iteration #3.  Calling sqlite ...
Updating paths
Done, current path total is: 18
Converged! The average length of the shortest paths is:  1.66666666667


In [5]:
# this is not part of the hw2, so that you can skipp it.
import numpy as np
p = np.ones((11,6), dtype=np.float)
for i in np.arange(1,11):
    p[i,0] = 0.15 + 0.85 * (p[i-1,1]/3) #PR(A) = 0.15 + 0.85 * PR(B/3)
    p[i,1] = 0.15 + 0.85 * (p[i-1,0]/3) #PR(B) = 0.15 + 0.85 * PR(A/3)
    p[i,2] = 0.15 + 0.85 * (p[i-1,0]/3 + p[i-1,1]/3) #PR(C) = 0.15 + 0.85 * (PR(A)/3 + PR(B)/3)
    p[i,3] = 0.15 + 0.85 * (p[i-1,0]/3 + p[i-1,1]/3 + p[i-1,2]) #PR(D) = 0.15 + 0.85 * (PR(A)/3 + PR(B)/3 + PR(C))
    p[i,4] = 0.15 + 0.85 * (p[i-1,3]) #PR(E) = 0.15 + 0.85 * PR(D)
    p[i,5] = 0.15 + 0.85 * (p[i-1,4]) #PR(F) = 0.15 + 0.85 * PR(E)
print(p)
print('at Iteration 10 the page rank value of every node is:')
print('previouse page Rank values from a - f:')
print(p[i-1,:])
print('Current page Rank values from a - f:')
print(p[i,:])

[[ 1.          1.          1.          1.          1.          1.        ]
 [ 0.43333333  0.43333333  0.71666667  1.56666667  1.          1.        ]
 [ 0.27277778  0.27277778  0.39555556  1.00472222  1.48166667  1.        ]
 [ 0.22728704  0.22728704  0.30457407  0.6407963   1.00401389  1.40941667]
 [ 0.21439799  0.21439799  0.27879599  0.53768395  0.69467685  1.00341181]
 [ 0.2107461   0.2107461   0.2714922   0.50846879  0.60703136  0.74047532]
 [ 0.20971139  0.20971139  0.26942279  0.50019116  0.58219847  0.66597665]
 [ 0.20941823  0.20941823  0.26883646  0.49784583  0.57516248  0.6448687 ]
 [ 0.20933516  0.20933516  0.26867033  0.49718132  0.57316895  0.63888811]
 [ 0.20931163  0.20931163  0.26862326  0.49699304  0.57260412  0.63719361]
 [ 0.20930496  0.20930496  0.26860992  0.49693969  0.57244408  0.6367135 ]]
at Iteration 10 the page rank value of every node is:
previouse page Rank values from a - f:
[ 0.20931163  0.20931163  0.26862326  0.49699304  0.57260412  0.63719361]
Current

In [6]:
# Q2
# Expected output: nodes table populated with pagerank results 
# Note:
# - keeping track of the old PR value helps find convergence, though it's not required in the
#   assignment, it will help you see how close the values end up becoming.
def initPageRank():
    dbQ2.executescript("""
        DROP TABLE IF EXISTS nodes;
        CREATE TABLE nodes (
            node varchar(200) PRIMARY KEY,
            prevpr real,
            currentpr real,
            outdegree integer
        );
    """);
    dbQ2.commit()
    return


def populatePageRank():
    dbQ2.executescript("""
        INSERT INTO nodes
        SELECT
            DISTINCT links.src,
            1,
            1,
            COUNT(links.dst)
        FROM
            links
        GROUP BY
            links.src;
               
        -- corner case #1 nodes with no outgoing edges
        INSERT INTO
            nodes
        SELECT
            DISTINCT links.dst,
            1,
            1,
            0
        FROM
            links
        WHERE
            links.dst NOT IN (
                SELECT node FROM nodes
            );    
            
        -- corner cases #2 nodes with no incoming edges
        UPDATE
            nodes
        SET
            currentpr = 0.15
        WHERE
            nodes.node not in (
                SELECT DISTINCT dst
                FROM links
            );
    """);
    dbQ2.commit()

    return


def updatePageRank():

    dbQ2.executescript(""" 
        -- TODO (hint: you might want a helper table to store intermediate results)
        
    
    """);
    dbQ2.commit()

    return

def topPageRank():
    
    dbQ2.executescript("""
    
        DROP VIEW IF EXISTS topPR;
        CREATE VIEW topPR AS
        SELECT *
        FROM nodes
        ORDER BY currentpr DESC
        LIMIT 100;
        
    """);
    dbQ2.commit()

    return

def printTopPageRank():
    # we need to create an cursor object to fetch results to this program
    #    which we didn't need to before since it was just updating the DB
    r = dbQ2.execute("SELECT * FROM topPR;").fetchall()
    for i in r:
        print i
    

def runPageRank(N=10):
    printNow('Initializing page rank')
    initPageRank()
    printNow('Populating PageRank')
    populatePageRank()
    for i in xrange(N):
        printNow('Updating PageRank for iteration {}'.format(i))
        updatePageRank()
        
    printNow('Running top pagerank!')        
    topPageRank()
    printNow('Done! Here are your results')        
    printTopPageRank()
    
    return


runPageRank()

Initializing page rank
Populating PageRank
Updating PageRank for iteration 0
Updating PageRank for iteration 1
Updating PageRank for iteration 2
Updating PageRank for iteration 3
Updating PageRank for iteration 4
Updating PageRank for iteration 5
Updating PageRank for iteration 6
Updating PageRank for iteration 7
Updating PageRank for iteration 8
Updating PageRank for iteration 9
Running top pagerank!
Done! Here are your results
('a', 1.0, 1.0, 3)
('b', 1.0, 1.0, 3)
('c', 1.0, 1.0, 1)
('d', 1.0, 1.0, 1)
('e', 1.0, 1.0, 1)
('f', 1.0, 1.0, 0)


In [8]:
# you should have ran the two previous cells before running the test!
# make sure also that you have set the db to part2test.db!
! chmod u+x test.sh
! ./test.sh

PASS q1
ERROR q2 output differed! See diffs/q2.csv
